In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [3]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [4]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [5]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [6]:
scenario =  """14. VerifyDescriptionFieldValidation"""

In [7]:
prompt_uc = """
Add a new book
Description
The creation page is designed to create a record of the new book in the database.

Main scenario:
User clicks the "Create new" button in the book list page;
Application displays form to enter book data;
User enters book data and presses "Submit" button;
If any data is entered incorrectly, incorrect data messages are displayed;
If entered data is valid, then record is adding to database;
If error occurs, then error message is displaying;
If new book record is successfully added, then list of books with added records is displaying.

Cancel operation scenario:
User clicks the "Create new" button in the book list page;
Application displays form to enter book data;
User may start entering the data into the form fields;
Before pressing "Submit" button user presses “Cancel” button;
Data doesn’t save in the database, then a list of books records is displayed to the user;
If the user selects the menu item "Books”, "Genres”, "Authors", "Home" or "API", the data will not be saved to the database, and the corresponding form with updated data will be opened.

When creating a new book, the following details are entered:
Title
Author
Genre
Description
Publication date
"""

In [8]:
BASELINE_GENERATE_TC = """
read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":{
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
  },
}
If there are more than one test case for this scenario, continue writing other test case in this form."""

In [9]:
use_case_directory_path = r"D:\GPT-testing\SpecificationData\MatchaEnglishWebsite"
test_scenario_directory_path = r"D:\Dissertation-GPT\evaluate\baseline 2\matcha\ts"
save_testcase_path = r"D:\Dissertation-GPT\evaluate\baseline 2\matcha\tc 2"

In [10]:

promptTestCase = [
{ "role": "system", "content": BASELINE_GENERATE_TC},
{ "role": "user", "content": scenario + "\n" + prompt_uc}
]
gpt_response = askJSON(promptTestCase, client, model)
print(gpt_response)


{
  "Test Case 1": {
    "testCaseName": "Verify Description Field Accepts Valid Input",
    "objective": "Ensure that the Description field accepts valid inputs without displaying incorrect data messages and the new book record is successfully added.",
    "testSteps": [
      "Step 1: Navigate to the book list page and click the 'Create new' button.",
      "Step 2: Fill in all the fields with valid data, including a valid description for the book.",
      "Step 3: Press the 'Submit' button."
    ],
    "expectedResult": "No incorrect data messages are displayed for the Description field, and the new book record is successfully added to the database. The user is redirected to the list of books with the newly added record visible."
  },
  "Test Case 2": {
    "testCaseName": "Verify Description Field Validation for Empty Input",
    "objective": "To verify that the application prompts the user with an incorrect data message when the Description field is left empty.",
    "testSteps": 